In [31]:
import re
import docx
import json

file_name = "107_2016_QH13_280693_highlighted.docx"

doc = docx.Document(file_name)

document_json = {
    "document" : []
}
# Put a place holder for later indexing
for para in doc.paragraphs:
	# if para.text is not \xa0
	if para.text != '\xa0':
		document_json["document"].append({
			"index" : "0.0.0.0",
			"text" : para.text
		})

document_data = document_json["document"]
    
def roman_to_int(s):
	rom_val = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
	int_val = 0
	for i in range(len(s)):
		if i > 0 and rom_val[s[i]] > rom_val[s[i - 1]]:
			int_val += rom_val[s[i]] - 2 * rom_val[s[i - 1]]
		else:
			int_val += rom_val[s[i]]
	return int_val

document_data_len = len(document_data)

for i in range(1, document_data_len):
	previous_element_index_list = document_data[i-1]["index"].split(".")

	previous_element_chapter_number = int(previous_element_index_list[0])
	previous_element_section_number = int(previous_element_index_list[1])
	previous_element_subsection_number = int(previous_element_index_list[2])
	previous_element_point_number = previous_element_index_list[3]
	element = document_data[i]
	# if element["text"] starts with "Chương" then it is a chapter, update the index
	if element["text"].startswith("Chương"):
		second_element = element["text"].split()[1]
		if (second_element[0] in ['I', 'V', 'X', 'L', 'C', 'D', 'M']):
			# if first character is a number then it is a chapter
		# chapter number is the first word right after "Chương"
			chapter_number = element["text"].split()[1]
			# right now the chapter number is roman numeral, convert it to arabic numeral
			chapter_number = roman_to_int(chapter_number)
			# update the index
			index_list = element["index"].split(".")
			index_list[0] = str(chapter_number)
			element["index"] = ".".join(index_list)
	elif element["text"].startswith("Điều") and element["text"].split()[1][0].isdigit():
		# section number is the first word right after "Điều", remove the dot at the end
		section_number = element["text"].split()[1][:-1]
		# update the index
		index_list = element["index"].split(".")
		index_list[0] = str(previous_element_chapter_number)
		index_list[1] = section_number
		element["index"] = ".".join(index_list)
	# else if element["text"] starts with a number and a dot then it is a subsection, update the index
	elif re.match(r"^\d+\.", element["text"]):
		# subsection number is the first word, remove the dot at the end
		subsection_number = element["text"].split(".")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[2] = subsection_number
		index_list[1] = str(previous_element_section_number)
		index_list[0] = str(previous_element_chapter_number)
		element["index"] = ".".join(index_list)
	# else if element["text"] starts with a character and a ) then it is a point, update the index
	elif re.match(r"^[a-zđê]\)", element["text"], re.IGNORECASE):
		# point number is the first character, remove the bracket at the end
		point_number = element["text"].split(")")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[3] = point_number
		index_list[2] = str(previous_element_subsection_number)
		index_list[1] = str(previous_element_section_number)
		index_list[0] = str(previous_element_chapter_number)
		element["index"] = ".".join(index_list)
	else:
		index_list = element["index"].split(".")
		index_list[0] = str(previous_element_chapter_number)
		index_list[1] = str(previous_element_section_number)
		if previous_element_section_number != 0 and previous_element_subsection_number == 0:
			index_list[2] = "1"
		elif previous_element_subsection_number != 0 and previous_element_point_number == 0:
			index_list[2] = str(previous_element_subsection_number)
			index_list[3] = str(previous_element_point_number)
		elif previous_element_point_number != 0:
			index_list[2] = str(previous_element_subsection_number)
			index_list[3] = str(previous_element_point_number)
		element["index"] = ".".join(index_list)

# save document_data to json file
with open(file_name[:-5] + '_indexed.json', 'w', encoding="utf-8") as json_file:
	json.dump(document_json, json_file, indent=4, ensure_ascii=False)	

In [7]:
with open('108_2016_QH13_303284_highlighted.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	print(data['document'][-1])

{'index': '0.0.0.0', 'text': '\xa0'}


In [13]:
test = '53.'
test[0].isdigit()

True